In [68]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import *
import tensorflow as tf
import numpy as np
from keras.models import Model
import keras
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard


In [69]:
base_model = ResNet50(include_top = False,weights = 'imagenet',input_shape = (224,224,3))


In [70]:
for layer in base_model.layers:
    layer.trainable = False

In [71]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2,activation='softmax')(x)


model = Model(inputs = base_model.input,outputs = x)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [72]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

__________________________________________________________________________________________________
add_159 (Add)                   (None, 7, 7, 2048)   0           bn5b_branch2c[0][0]              
                                                                 activation_484[0][0]             
__________________________________________________________________________________________________
activation_487 (Activation)     (None, 7, 7, 2048)   0           add_159[0][0]                    
__________________________________________________________________________________________________
res5c_branch2a (Conv2D)         (None, 7, 7, 512)    1049088     activation_487[0][0]             
__________________________________________________________________________________________________
bn5c_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5c_branch2a[0][0]             
__________________________________________________________________________________________________
activation

In [73]:
# adam = Adam(lr = 0.00003)
# model.compile(loss='categorical_crossentropy',optimizer = adam, metrics = ['accuracy'])

In [74]:

data_generator = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   rotation_range = 90,
                                   brightness_range = [1.0,1.5],
                                   horizontal_flip = True,
                                   validation_split=0.2,
                                   fill_mode="nearest",
                                   preprocessing_function=preprocess_input)
valAug = ImageDataGenerator(rescale = 1./255,preprocessing_function=preprocess_input)

 

 

In [75]:


train_generator = data_generator.flow_from_directory(
        './breast-histopathology-images/8867/',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 300x300
        batch_size=32,
        seed = 42,
        shuffle = True,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = valAug.flow_from_directory(
        './breast-histopathology-images/8914/',
        target_size=(224, 224),
        batch_size=32,
        seed = 42,
        shuffle = True,
        class_mode='categorical')


Found 934 images belonging to 2 classes.
Found 350 images belonging to 2 classes.


In [76]:
# learn_control = ReduceLROnPlateau(monitor='val_acc', patience=5)

# # Checkpoint
# filepath="bests.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# from keras.callbacks import EarlyStopping, ModelCheckpoint

# es_callback = EarlyStopping(monitor='val_acc', patience=5, mode='max')
# mc_callback = ModelCheckpoint(filepath='resbestweight.h5', monitor='val_acc', save_best_only=True, mode='max')
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


In [ ]:
# TRAINING

hist = model.fit_generator(
    train_generator,
    steps_per_epoch= 50,
    epochs=15,
    validation_data=validation_generator,
    validation_steps = 50,
    verbose = 2,
    callbacks=[es_callback]
)



Epoch 1/15


In [ ]:
import matplotlib.pyplot as plt
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

In [ ]:
plt.plot(acc,label='accuracy')
plt.plot(val_acc,label='val_acc')
plt.legend()
plt.show()


In [ ]:
plt.plot(loss,label='loss')
plt.plot(val_loss,label= 'val_loss')
plt.legend()
plt.show()